In [ ]:
import tf_utils
from utils import *

from tensorflow_addons.metrics import CohenKappa
from tensorflow.keras.models import Model, load_model
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, Activation, concatenate, GlobalAveragePooling2D, Dense, ZeroPadding2D, AveragePooling2D

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Nadam, SGD

import tensorflow as tf

import math

In [ ]:
block_number = 0
transition_number = 0
K=12 
m=0.5

def transition_layer(input_block):
    global transition_number
    
    tran_name = 'tran' + str(transition_number) + '_'
    
    transition_number += 1 
    x = BatchNormalization(momentum=bn_momentum, name=tran_name + 'bn_t')(input_block)
    x = Activation('relu', name=tran_name + 'activation_t')(x)
    x = Conv2D(
        filters=int(x.shape[ -1 if keras.backend.image_data_format() == "channels_last" else 1 ]*m), 
        kernel_size=1, 
        padding='same',
        name=tran_name + 'conv_t',
        kernel_initializer=initializer
    )(x)
    x = AveragePooling2D(pool_size = 2, strides = 2)(x)
    return x

def dense_block(input_block, blocks):    
    global block_number
    x = input_block
    
    for b in range(blocks):
        block_number += 1
        block_name = 'block' + str(block_number) + '_'

        y = BatchNormalization(momentum=bn_momentum, name=block_name + 'bn_1')(x)
        y = Activation('relu', name=block_name + 'activation_1')(y)
        y = Conv2D(
            filters=K*4, 
            kernel_size=1,
            padding='same', 
            name=block_name + 'conv_1',
            kernel_initializer=initializer
        )(y)

        y = BatchNormalization(momentum=bn_momentum, name=block_name + 'bn_2')(y)
        y = Activation('relu', name=block_name + 'activation_2')(y)
        y = Conv2D(
            filters=K, 
            kernel_size=3, 
            padding='same', 
            name=block_name + 'conv_2',
            kernel_initializer=initializer
        )(y)
        
        x = concatenate([y, x])
    return x

bn_momentum = 0.9
initializer='he_uniform'

input_shape = (320, 320)

MODEL_PATHS = data_paths('densenet_2')

#mirrored strategy for distributed training
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():

    input_img = Input(shape=(*input_shape, 1), name='input')

    x = ZeroPadding2D(3, name='zero_padding_1')(input_img)
    x = BatchNormalization(name='bn_first')(x)
    x = Activation('relu', name='activation_first')(x)
    x = Conv2D(filters=64, kernel_size=7, strides=2, name='conv_first', kernel_initializer=initializer)(x)

    x = ZeroPadding2D(1, name='zero_padding_2')(x)
    x = MaxPool2D(3, strides=2, name='max_pooling')(x)
    x = BatchNormalization(name='bn_first_pool')(x)
    x = Activation('relu', name='activation_first_pool')(x)

    #[6, 12, 32, 32]
    x = dense_block(x, 6)
    x = transition_layer(x)
    x = dense_block(x, 12)
    x = transition_layer(x)
    x = dense_block(x, 32)
    x = transition_layer(x)
    x = dense_block(x, 32)

    x = BatchNormalization(name='bn_last')(x)
    x = Activation('relu', name='activation_last')(x)

    x = GlobalAveragePooling2D(name='GAP')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=[input_img], outputs=[output])

    optimizer = SGD(learning_rate=1e-2, momentum=0.9)

    model.compile(
        optimizer=optimizer,
        metrics=[CohenKappa(num_classes=2), 'accuracy'],
        loss='binary_crossentropy',
    )

In [ ]:
model.summary()

In [ ]:
set_note(MODEL_PATHS, 
        'DenseNet-169, input shape 320, aug 2, vaug 1, SGD 0.01, decay 0.95 every 10 e, mom 0.9 with lrs, bn_m 0.9, batch size 64, mirrored',
         summary_append=model
        )

model.save(MODEL_PATHS['init'])
print('Created: ' + MODEL_PATHS['init'])

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)